In [14]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup

In [17]:
CHEMBL = "https://www.ebi.ac.uk/chembl/beta/compound_report_card/CHEMBL1/"

driver = webdriver.Edge()
driver.get(CHEMBL)
content = driver.page_source.encode('utf-8').strip()

CHEMBLsoup = BeautifulSoup(content, "html.parser")
table = CHEMBLsoup.find_all("table", class_="compact-table hide-on-small-only")[0]
table_body = table.find("tbody")
row = table_body.find_all('tr')[1]
col = row.find("td", class_="BCK-CanonicalSmiles")
SMILES = CHEMBLsoup.find("input", id_="CompReps-canonicalSmiles").value()
print(SMILES)
    
    #CHEMBL = "https://www.ebi.ac.uk/chembl/beta/compound_report_card/CHEMBL"

WebDriverException: Message: 'MicrosoftWebDriver.exe' executable needs to be in PATH. Please download from http://go.microsoft.com/fwlink/?LinkId=619687
